In [ ]:
import os
current_folder = os.getcwd()
project_path = os.path.join(current_folder, "data/travel-service-dev")

In [ ]:
!pip install openai

In [ ]:
import openai
from datetime import datetime, time
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
gpt_client = openai.OpenAI()

In [ ]:
def query_gpt(
    prompt: str,
    #lm: str = 'gpt-3.5-turbo-1106',
    lm: str = 'gpt-4-1106-preview',
    temperature: float = 0.0,
    max_decode_steps: int = 512,
    seconds_to_reset_tokens: float = 30.0,
) -> str:
  while True:
    try:
      raw_response = gpt_client.chat.completions.with_raw_response.create(
        model=lm,
        max_tokens=max_decode_steps,
        temperature=temperature,
        messages=[
          {'role': 'user', 'content': prompt},
        ]
      )
      completion = raw_response.parse()
      return completion.choices[0].message.content
    except openai.RateLimitError as e:
      print(f'{datetime.datetime.now()}: query_gpt_model: RateLimitError {e.message}: {e}')
      time.sleep(seconds_to_reset_tokens)
    except openai.APIError as e:
      print(f'{datetime.datetime.now()}: query_gpt_model: APIError {e.message}: {e}')
      print(f'{datetime.datetime.now()}: query_gpt_model: Retrying after 5 seconds...')
      time.sleep(5)

In [ ]:
from projectfiles import ProjectFiles

In [ ]:
pf = ProjectFiles(root_path=project_path, prefix_list=["src/main/java"], suffix_list=[".java"])

In [ ]:
codefiles = pf.get_files_of_project()

In [ ]:
prompt_shorten_template = """
You are a world class Java developer. You are given a Java program to maintain. You need to read the code and write notes.
The notes should be short, concise and to the point.
Make sure to include the following points:
- The purpose of the code
- The functionality of the code
- The important classes and methods used in the code

Just return the notes. DO NOT explain your reason.

File Name: {}

Package: {}

Code:

{}

"""

In [ ]:
def code_gisting(code_file, verbose=True) -> str:
    with open(code_file.path, 'r') as file:
        code = file.read()
    prompt = prompt_shorten_template.format(code_file.filename, code_file.package, code)
    summary = query_gpt(prompt)
    if verbose:
        print(f"Summary of the code file {code_file.filename}: {summary}")
    return summary

In [ ]:
for file in codefiles:
        print(file.filename, file.package)
        notes = code_gisting(file)
        file.set_summary(notes)

In [ ]:
prompt_pacakge_notes_template = """
You are a world class Java developer. You are given a legacy code base to maintain. 
You already read the code and wrote notes about the code files. Now you need to write notes about the packages.
Below you are given the notes of the code files in the package, as well as the notes of its sub-packages.
Make sure to include the following points:
- The purpose of the package

Just return the notes.
DO NOT explain your reason.

Package Name: {}

Notes of Sub Packages: 
{}

Notes of Direct Child Files: 
{}

"""

In [ ]:
def real_package_gisting(package, subpackage_notes, filenotes):
    print(f"\n\nchecking LLM on package: {package}")
    prompt = prompt_pacakge_notes_template.format(package, subpackage_notes, filenotes)
    notes = query_gpt(prompt)
    return notes

In [ ]:
pf.package_gisting = real_package_gisting
pf.package_structure_traverse(packages=None, action_file=pf.execute_on_file, action_package=pf.execute_on_package, is_bottom_up=True)

In [ ]:
print(pf.package_notes)

In [ ]:
prompt_continue_template = """
You are a world class Java developer, assigned to work on an existing project and accomplish a task. 

Here is the task you need to accomplish:
{}

Below is the Java project structure for your reference:
{}

You need to write the steps to accomplish the task. For now only focus on development tasks only. Do not focus on testing, deployment, or other tasks.

Since you are new to this project, if you have questions or need help, you are encouraged to ask for help, in below format:
[I need access files: <file1 name>,<file2 name>,<file3 name>]
[I need info about packages: <package1 name>,<package2 name>,<package3 name>]

ONLY ask for the file or package that you know exists. If you are unsure, ask for info of project.
[I need to know the project structure]

If you need more information, please ask for it in the following format:
[I need clarification about <what you need clarification about>]

You are known for deep thinking and detail oriented but also occasionally suffer from memory loss, therefore you likie writing down your own reasoing and train of thoughts. 
This way I can ensure to remind you of the reasoning and thoughts in the future.

Your end goal is to write the steps in a clear and concise manner, for example
[Step 1]
[Step 2]
...


below are your notes from previous research of the project:
{}

{}

Only stop when you are very sure of the steps. If you are not sure, ask for more info of the files or packages and your reasoning.
"""

In [ ]:
from typing import Tuple

In [ ]:
def ask_continue(last_response, pf, past_additional_reading) -> Tuple[str, str, bool]:
    projectTree = pf.to_tree()
    additional_reading = ""
    for line in last_response.split("\n"):
        if line.startswith("[I need access files:"):
            # for example [I need access files: <file1 name>,<file2 name>,<file3 name>]
            file_names = line.split(":")[1].strip().rstrip("]").split(",")
            print(f"LLM needs access to files: {file_names}")
            for file_name in file_names:
                # clean it
                file_name = file_name.strip()
                file = pf.find_codefile_by_name(file_name, package=None)
                if file:
                    additional_reading += f"You asked about file: {file.filename}\n"
                    additional_reading += f"{file.summary}\n"
                    # now let's get the file content, since we have the path
                    with open(file.path, "r") as f:
                        file_content = f.read()
                        additional_reading += f"Below is the file Content:\n {file_content}\n"
                else:
                    additional_reading += f"File {file_name} does not exist! Please ask for the correct file or packages! I am very disappointed!\n"
        elif line.startswith("[I need info about packages:"):
            # example [I need info about packages: <package1 name>,<package2 name>,<package3 name>]
            package_names = line.split(":")[1].strip().rstrip("]").split(",")
            print(f"Need more info of package: {package_names}")
            for package_name in package_names:
                # clean it
                package_name = package_name.strip()
                # first get the notes of the package
                notes = pf.find_notes_of_package(package_name)
                if notes:
                    additional_reading += f"Info about package: {package_name} :\n {notes}\n\n"
                # now let's get the sub-packages and code files
                subpackages, codefiles = pf.find_subpackages_and_codefiles(package_name)
                if subpackages:
                    additional_reading += f"this package has below sub packages: {subpackages}\n\n"
                codefilenames = [f.filename for f in codefiles]
                if codefilenames:
                    additional_reading += f"this package has files: {codefilenames}\n\n"
        elif line.startswith("[I need to know the project structure]"):
            print("LLM needs to know the project structure")
            additional_reading += f"Info about project structure: \n{projectTree}\n"
        elif line.startswith("[I need clarification about"):
            # extract the part between '[I need clarification about' and ']'
            what = line.split("[I need clarification about")[1].split("]")[0]
            print(f"LLM needs more information: \n{what}")
            # ask user to enter manually through commmand line
            human_response = input("Please enter the additional reading for the LLM\n")
            additional_reading = f"Regarding {what}, {human_response}\n"
        elif line.startswith("[I need"):
            print(f"LLM needs more information: \n{line}")
            # ask user to enter manually through commmand line
            human_response = input("Please enter the additional reading for the LLM\n")
            additional_reading = f"Question:{line}\nAnswer: {human_response}\n"
        else:
            pass
    if last_response=="" or additional_reading:
        # either the first time or the last conversation needs more information
        if last_response=="":
            # pretty print the package_notes into a big string
            package_notes_str = ""
            for package, notes in pf.package_notes.items():
                package_notes_str += f"Package: {package}\nNotes: {notes}\n\n"
            last_response = package_notes_str
            
        prompt = prompt_continue_template.format(ask, projectTree, last_response, "Below is the additional reading you asked for:\n" + past_additional_reading + "\n\n" + additional_reading)
        # request user click any key to continue
        input("Press Enter to continue to send message to LLM ...")
        response = query_gpt(prompt)
        return response, additional_reading, False
    else:
        print("the LLM does not need any more information, so we can end the conversation")
        return last_response, None, True

In [ ]:
ask = "add a new endpoint /admin/healthcheck that returns OK for any requests"

In [ ]:
max_rounds = 8
print(f"Ask LLM: {ask} max_rounds: {max_rounds}")
# looping until the user is confident of the steps and instructions, or 8 rounds of conversation
i = 0
past_additional_reading = ""
doneNow = False
additional_reading = ""
last_response = ""
while True and i < max_rounds:
    response, additional_reading, doneNow = ask_continue(last_response, pf, past_additional_reading=past_additional_reading)
        
    # check if the user is confident of the steps and instructions
    if doneNow:
        print(response)
        break
    else:
        past_additional_reading += ("\n" + additional_reading)
        last_response = response
        i += 1